In [149]:
#change original format
import re
import os
import pandas as pd

def change_original_column_name(path = r"data\excel_output\original_data.xlsx", outpath = r"data\excel_output\formatted_data.xlsx"):
    out = pd.read_excel(path) #read excel

    # for i in range (0,len(out)-1,2): #remove name row
    #     out.iloc[i:i+1,1:] = out.iloc[i+1:i+2,1:]
    # out = out[out['Name'].notna()]

    out = out.rename(columns={'nad4L':'nad4l', 'tA':'trnA', 'tC':'trnC', 'tD':'trnD', 'tE':'trnE', #change column name
                            'tF':'trnF', 'tG':'trnG', 'tH':'trnH', 'tL1':'trnL1', 'tL2':'trnL2', 
                            'tM':'trnM', 'tN':'trnN', 'tP':'trnP', 'tQ':'trnQ', 'tR':'trnR', 
                            'tS1':'trnS1', 'tS2':'trnS2', 'tT':'trnT', 'tV':'trnV', 'tW':'trnW', 
                            'tY':'trnY'})

    out.to_excel (outpath, index = False, header=True)
change_original_column_name()

In [142]:
def file_to_df(path, dir = "data\\fasta_file\\"):
    data_dict = {}

    with open(dir + path,"r") as obj:
        data = obj.read()

    datalist = re.findall("[>][^>]+", data)

    for item in datalist:
        a = re.findall("_([a-zA-Z0-9.]{3,})", item)[-1]
        gene_name = item.splitlines()[0]
        i = ''.join(item.splitlines()[1:])
        data_dict[a] = []
        data_dict[a].append(gene_name)  #####original format
        data_dict[a].append(i)
        filename = re.match("(\w+)(\.)",path)
        df = pd.DataFrame(data_dict)
        df['Name'] = ">" + filename.group(1)
        df = df.sort_index(axis = 1)
        df.iloc[1,0] = ""  #####original format
        # print(gene_name)
    return df


In [143]:
def get_fasta_file_data(dir = "data\\fasta_file"):
    filelist = os.listdir(dir)
    templist = []
    for file in filelist:
        templist.append(file_to_df(file,dir+"\\"))
    return templist

In [144]:
def combine_data_frame(dir = "data\\fasta_file"):
    templist = get_fasta_file_data(dir)
    result = templist[0]
    n = 0
    while n < len(templist)-1:
        n += 1
        result = pd.concat([templist[n],result],ignore_index=True) #combine data and sort column name
    return result


In [147]:
def write_data_to_excel(excel_path,data): #add the new combined data to original excel
    original_data = pd.read_excel(excel_path, index_col=False)
    original_data = pd.concat([original_data,data],ignore_index=True)
    original_data.to_excel("data\excel_output\output.xlsx",index = False) 

In [153]:
##main (change format)
original_excel_path = r"data\excel_output\original_data.xlsx"
formatted_excel_path = r"data\excel_output\formatted_data.xlsx"
fasta_file_dir = r"data\fasta_file"

##change format
change_original_column_name(original_excel_path,formatted_excel_path)
write_data_to_excel(formatted_excel_path,combine_data_frame(fasta_file_dir))


In [ ]:
##main (original format)
# excel_path = r"data\excel_output\希望的格式.xlsx"
# fasta_file_dir = r"data\fasta_file"
# write_data_to_excel(excel_path,combine_data_frame(fasta_file_dir))


In [156]:
# def excel_to_fasta(excel_path,output_fasta_path):
excel_path = "data\excel_output\output.xlsx"
df = pd.read_excel(excel_path)
df

,Name,atp6,atp8,cob,cox1,cox2,cox3,nad1,nad2,nad3,...,trnR,trnS1,trnS2,trnT,trnV,trnW,trnY,18S,5.8S,28S
0,>KF590525_LWW1204387_Athyma_selenophora,>LWW1204387_Athyma_pravara40394713+atp6,>LWW1204387_Athyma_pravara38784042+atp8,>LWW1204387_Athyma_pravara1042811606+cob,>LWW1204387_Athyma_pravara14313056+cox1,>LWW1204387_Athyma_pravara29283758+cox2,>LWW1204387_Athyma_pravara47245527+cox3,>LWW1204387_Athyma_pravara1163212606nad1,>LWW1204387Athyma_pravara,>LWW1204387Athyma_pravara,...,>LWW1204387_Athyma_pravara60076069+trnR(tcg),>LWW1204387_Athyma_pravara61346194+trnS1(gct),>LWW1204387_Athyma_pravara1158411648+trnS2(tga),>LWW1204387_Athyma_pravara97679831+trnT(tgt),>LWW1204387_Athyma_pravara1399814062trnV(tac),>LWW1204387Athyma_pravara_tTrp,>LWW1204387_Athyma_pravara13951460trnY(gta),NaN,NaN,NaN
1,NaN,ATGATAAATAATTTATTTTCAATTTTTGATCCTTCCACTAATATTT...,ATTCCTCAAATAATACCAATTAATTGAATTTTATCTTTTATTTTTT...,ATGATAAATTTATATAAACCTATTCGTAAAACTCACCCAGTAATTA...,CGAAAATGACTTTTTTCTACAAATCATAAGGATATTGGAACTTTAT...,ATGGCAACATGATCTAATCTTAATTATCAAAATAGAGCTTCTCCTT...,ATGACTCTTAATCATAATCATCCATATCATTTAGTAGATTATAGAC...,ATGATAATACAAGATTTATATAATATTTTTATCGGGATTTTAATTT...,ATTATTTTAATAACTAACTCTAATAAAATATTTTTTTTATTTATTT...,ATAATTAATTTAATAATAATAATATCTTTAATTATAATTATTATTT...,...,AAATAAGAAGCAATTTGCATTTAATTTCGACTTAAAAGATAGAGTT...,GAAATATATAATAATTAAGCTGCTAACTTAATTTATAGTGAATAAA...,AATTAATGAGCTTGTTTAAGCATTTGTTTTGAAAACTTAAGAAAGA...,GTTTTTATAGTTTAATAAAAAACTTTGGTCTTGTAAATCAAAAATA...,CAAATTAGAGCTTGTTAAGTATTTCATTTACATTGAAAAGAAATTA...,AAGGTTTTAAGTTAATATAAACTAATAATCTTCAAAATTATATATA...,AATAAAATGGCTGAGGTATAAGCGATAAATTGTAAATTTATTAACA...,NaN,NaN,NaN
2,>N4755_Limenitis_disjuncta,>N4755Limenitis_moltrechti_40414706_+_atp6,>N4755Limenitis_moltrechti_38804041_+_atp8,>N4755Limenitis_moltrechti_1047611558_+_cob,>N4755Limenitis_moltrechti_14592973_+_cox1,>N4755Limenitis_moltrechti_30613723_+_cox2,>N4755Limenitis_moltrechti_47275512_+_cox3,>N4755Limenitis_moltrechti_1169512618__nad1,>N4755Limenitis_moltrechti_2871159_+_nad2,>N4755Limenitis_moltrechti_55885929_+_nad3,...,>N4755Limenitis_moltrechti_60046065_+_trnR(tcg),>N4755Limenitis_moltrechti_61356195_+_trnS1(gct),>N4755Limenitis_moltrechti_1160511669_+_trnS2(...,>N4755Limenitis_moltrechti_97909855_+_trnT(tgt),>N4755Limenitis_moltrechti_1402114085__trnV(tac),>N4755Limenitis_moltrechti_12691336_+_trnW(tca),>N4755Limenitis_moltrechti_13941459__trnY(gta),>N4755_18S,>N4755_5.8S,>N4755_28S
3,NaN,ATGATAAATAATTTATTTTCAATTTTTGACCCTTCTACTAATATTT...,ATTCCACAAATAATACCTATTAATTGAATTCTTTCATTAATCTTTT...,ATGATAAACTTATATAAACCTATTCGTAAAACTCATCCAGTATTTA...,TTGCGAAAATGACTTTTTTCAACAAATCATAAAGATATTGGAACTT...,ATGGCAACATGATCTAATTTTAATTATCAAAATAGAGCTTCCCCAT...,ATGACAACTAATTATAATCACCCTTATCATTTAGTAGATTATAGCC...,ATGATAATACAAGATTTATATAATATAATTATAGGGGGTTTAATTT...,ATTCTTTTAATACTTAATTCTAATAAAATATTTTTTTTATTTATTT...,ATTATTTTAATAATAATAATATCTTTTATTATAATCATTATTTCTA...,...,AAATAAGAAGCAATTTGCATTTAATTTCGACTTAAAAGATAGAGTT...,GAAATATATAATAATTAGGCTGCTAACTTAATTTTTAGTGAATAAA...,AATTAATGAGCTTGTATAAGCATTTGTTTTGAAAACTTAAGAAAGA...,GTTTTTATAGTTTAATAAAAAACTTTGGTCTTGTAAATCAAAAATA...,CAAATTAGAGCTTGATAAGTATTTCATTTACATTGAAAAGAAATTA...,AAGGTTTTAAGTTAATTATAAACTAATAATCTTCAAAATTATTTAT...,AATAAAATGGCTGAGTTATAAGCGATAAATTGTAAATTTATTAACG...,CAGCTCCCTGGTTGATCCTGCCAGTAGTTATATGCTTGTCTCAAAG...,AAACCACTACCCTGGACGGTGGATCACTAGGCTCGCGGGTCGATGA...,GCATATTAGTAAGCGGAGGAAAAGAAACTAACCAGGATTTCCTTAG...
4,>17LW02028_Athyma_lysanias,>17LW02028_Tarattia_bruijni_40334698_+_atp6,>17LW02028_Tarattia_bruijni_38724033_+_atp8,>17LW02028_Tarattia_bruijni_1042411506_+_cob0,>17LW02028_Tarattia_bruijni_14552969_+_cox1,>17LW02028_Tarattia_bruijni_30573719_+_cox2,>17LW02028_Tarattia_bruijni_47145499_+_cox3,>17LW02028_Tarattia_bruijni_1165012555__nad1,>17LW02028_Tarattia_bruijni_2771149_+_nad2,>17LW02028_Tarattia_bruijni_55725916_+_nad3,...,>17LW02028_Tarattia_bruijni_59956057_+_trnR(tcg),>17LW02028_Tarattia_bruijni_61216181_+_trnS1(gct),>17LW02028_Tarattia_bruijni_1155811624_+_trnS2...,>17LW02028_Tarattia_bruijni_97429805_+_trnT